# Udacity Data Analyst Nanodegree

# P3: Wrangle Open Street Map Data
17.03.2018 <br>
Author: Diogo Adão e Silva

## Context
-------
This notebook is my project 'Wrangle Open Street Map Data' submission. 'Data wrangling' is what the process of collecting and cleaning data is often called. Having completed the project means I am able to:
- Assess the quality of the data for validity, accuracy, completeness, consistency and uniformity.
- Parse and gather data from an xml file.
- Process data from a very large file that can be cleaned programmatically.
- Store, query, and aggregate data using SQL.

[OpenStreetMap (OSM)](https://www.openstreetmap.org/) is built by a community of mappers that contribute and maintain data about roads, buildings, restaurants, railway stations and much more, all over the world. Since any person can register on OSM and start editing a map, it is only natural that some of the data is not valid, accurate, complete, consistent or uniform.

## Map Area
-----
**Cologne, Germany**

* https://www.openstreetmap.org/relation/62578#map=10/50.8818/7.1823

OSM data is availabel in OSM XML format. To export the data I used [Mapzen's Metro Extracts](https://mapzen.com/documentation/metro-extracts/), which unfortunately closed at the end of January 2018. Mapzen offered raw OSM data clipped to the rectangular region surrounding a particular city or region of interest. My extract was exported on 05.12.2017.

I lived in Cologne between May 2013 and June 2017. Having enjoyed living there, I was interested in cleaning its map data.

## Understanding OSM's model of the physical world
------
The first look at the data was through the Unix less command in terminal, since opening the file in a text editor is not recommended due to its very large size. Using [create_sampleosm.py](https://github.com/DiogoAdaoeSilva/DAND/blob/master/p3/create_sampleosm.py), a small sample of the data was generated to make it easier to audit and iterate through the data.
### Elements
With the [mapparser.py](https://github.com/DiogoAdaoeSilva/DAND/blob/master/p3/mapparser.py) script a dictionary with the different elements and their respective number of occurences within the sample dataset was generated:
```python
{'node': 19900, 'nd': 29858, 'member': 505, 'tag': 20011, 'relation': 48, 'way': 3885, 'osm': 1}```
In the OSM data model, the three main mapping elements are: **nodes**, which represent points such as a local business; **ways**, which are linear features like roads or area boundaries like buildings; and **relations**, which explain how multiple elements can be connected, for e.g. multiple nodes that belong to a bus route.
### Tag elements
Tags are child elements of nodes, ways and relations, and consist of two attributes, a key 'k' and a value 'v', that describe the meaning of the particular element to which they are attached. The script [viewtagtypes.py](https://github.com/DiogoAdaoeSilva/DAND/blob/master/p3/viewtagtypes.py) allows us to have a look at the most common tags in the sample dataset:
```python
[('building', 2960),
 ('addr:street', 1730),
 ('addr:city', 1728),
 ('addr:postcode', 1725),
 ('addr:housenumber', 1722),
 ('addr:country', 1700),
 ('building:use', 1374),
 ('building:levels', 737),
 ('highway', 718),
 ('name', 486),
 ('source', 383),
 ('amenity', 181),
 ('roof:shape', 179),
 ('surface', 159)] ```

## Auditing data
-----
The functions used for auditing the following fields are found in the [audit.py](https://github.com/DiogoAdaoeSilva/DAND/blob/master/p3/audit.py) file.
### Street names
Using regular expressions, street names' endings and beginnings were audited with the [audit.py](https://github.com/DiogoAdaoeSilva/DAND/blob/master/p3/audit.py) script to check if there were inconsistencies or abbreviations, such as 'str.' instead of 'Straße'.
### Phone numbers
Phone numbers were matched against a standard pattern. According to [OSM wiki page](https://wiki.openstreetmap.org/wiki/DE:Key:phone), the correct international phone number format is `<country code> <3 digit area code> <6-8 digit number>`, where +49 is Germany's country code.
### Postcodes
Postcodes were compared to the official list of [postcodes belonging to Cologne](https://www.suche-postleitzahl.org/koeln-plz-50667-51467.4c37).

## Problems encountered in the map
-----
- Phone numbers with incorrect pattern. Some examples:
```python
{'+49 (0) 2203/9544 - 0', '+49 2203 9779052', '+49-0221-99205379', '0176 - 32 21 90 30', '022116913699'}
  ```
- Postcodes not belonging to Cologne: `{'50354','51467','51427','51469'}`. These postcodes belong to cities nearby Cologne, which were also captured by Mapzen's Metro Extract. <p>

- No problems where encountered with street names. All beginnings and endings of the street names where checked against a regular expression containing a list of common street types in Germany:
```python
pattern = r'''
^(Am\b|Auf\sdem\b|Auf\sder\b|Im\b|In\sder\b|An\b|Zum\b|Zur\b)| # match beginnings of street names
(straße|\bStraße|-Straße|\bWeg|weg|-Weg|platz|-Platz|\bPlatz| # match end of street names
gasse|\bRing|ring|\bHof|hof|
\bAllee|-Allee|allee|wall|\bWall|markt|gürtel)$'''```
Although cases of 'straße', 'Straße' and '-Straße' were found, this is not a case of inconsistency, because in German these use cases all represent the actual street name.

## Data cleaning
-----

In this step, by running the [data.py](https://github.com/DiogoAdaoeSilva/DAND/blob/master/p3/data.py) script, the problems encountered in the map were cleaned and data was converted from XML to CSV format. 

- Cleaning phone numbers:

```Python
# Regex to match correct phone number format
phone_re = re.compile(r'^\+49\s\d{3}\s\d{6,8}$')

def update_phone(phone_num):
  m = phone_re.search(phone_num)
  if not m:
    # remove all dashes
    if '-' in phone_num:
      phone_num = re.sub('-', '', phone_num)
    # remove all brackets
    if '(' in phone_num or ')' in phone_num:
      phone_num = re.sub('[()]', '', phone_num)
    # remove all slashes
    if '/' in phone_num:
      phone_num = re.sub('/','',phone_num)
    # remove all spaces
    if ' ' in phone_num:
      phone_num = re.sub(' ','', phone_num)
    # insert + sign if needed
    if re.match(r'49', phone_num):
      phone_num = '+' + phone_num
    # insert country code if needed
    if re.match(r'\w+', phone_num):
      phone_num = '+49' + phone_num
    # remove 0 from regional code
    if re.search(r'^\+490', phone_num):
      p = re.search(r'^(\+49)(0)', phone_num)
      phone_num = re.sub(r'^\+49(0)', p.group(1), phone_num)
    # insert space after country code and regional code
    if re.match(r'^(\+49)([1-9]\d{2})', phone_num):
      p = re.compile(r'(\+49)([1-9]\d{2})')
      phone_num = p.sub(r'\1 \2 ', phone_num)
    # exclude all numbers which even after all the above corrections still do not match the desired pattern
    # this will correct for number length (too long or too short) and for special characters such as ';' or ',' or '#'
    if re.search(r'^\+49\s\d{3}\s(\d{6,8})$', phone_num) is None:
      return
    
  return  phone_num```
  
  
- cleaning postcodes: Postcodes which did not match any of the postcode values associated with Cologne were skipped, i.e., incorrect postcodes were not exported to the csv files and therefore not imported to the database. If the postcode did not match the regular expression **r'^\d{5}$'**, it was skipped . If it matched the regular expression but it was not one of the postcodes associated with Cologne, then the whole element associated with that postcode was skipped.

## Importing to database
----
Finally, using the [createdb.py](https://github.com/DiogoAdaoeSilva/DAND/blob/master/p3/create_db.py) file, the cleaned .csv files were imported into a SQL database using a given schema. The osm.db database has five tables: nodes, nodes_tags, ways, ways_tags and ways_nodes.

### Inspecting cities

Having removed postcodes not associated with Cologne in the previous cleaning stage, most elements that have a key 'city' in them should have the value 'Köln' associated with that key. However, it is worth checking if some other cities made it through the cleaning stage.

```Sql
SELECT tags.value, COUNT(*) as count 
FROM (SELECT * FROM nodes_tags UNION ALL 
      SELECT * FROM ways_tags) tags
WHERE tags.key = 'city'
GROUP BY tags.value
ORDER BY count DESC;```

```
Köln|123108
50968|1
Bergisch Gladbach|1
Cologne|1
Hürth|1
Köln Porz-Eil|1
Köln Porz-Urbach|1
Köln-Nippes|1
```

Indeed, out of 123115 entries of the key 'city', seven do not have the correct value 'Köln' and will be removed. 
```Sql
DELETE FROM nodes_tags WHERE key = 'city' and value != 'Köln';
DELETE FROM ways_tags WHERE key = 'city' and value != 'Köln';```
### Inspecting phone numbers
The update_phone function used for cleaning phone numbers performed well when applied to the sample dataset. But how does it hold against the entire dataset?

```Sql
SELECT tags.value
FROM (SELECT * FROM nodes_tags UNION 
      SELECT * FROM ways_tags) tags
WHERE tags.key = 'phone' and tags.value NOT LIKE '+49 ___ %';```

Unfortunately this query returned several 'dirty' phone number formats. Here a couple of examples: 0221 510 91-0, +49 2234 1856-0, +49-221-2578555;+49-221-2578700, + 49 (0) 221 - 27 25 19 20.<p>
Why was the cleaning not 100% successful?

Testing the update_phone function with some of the'dirty' phone numbers showed that it was working as expected. The problem, therefore, had to be that the update_phone function was not being called in every instance a tag element had a phone number. This was confirmed, by finding that some phone numbers had a key 'contact:phone', which had not been taken into account. The shape_element function, where update_phone is called, was accordingly corrected to apply the update_phone function to any tag element that has the key 'phone' or the key 'contact:phone'.

## Data Overview
----
### File sizes
```
cologne_germany.osm ......... 402 MB
osm.db ...................... 223 MB
nodes.csv ................... 127 MB
nodes_tags.csv .............. 8.3 MB
ways.csv .................... 17.3 MB
ways_nodes.csv .............. 54.5 MB
ways_tag.csv ................ 43.8 MB
```
### Number of unique users
```Sql
sqlite> SELECT COUNT(*)
        FROM (SELECT uid FROM nodes UNION SELECT uid FROM ways);
```
2099
### Number of nodes
```SQL
sqlite> SELECT COUNT(id) FROM nodes;```
1591503
### Number of ways
```SQL
sqlite> SELECT COUNT(id) FROM ways;```
298404
### Top 10 users

```SQL
sqlite> SELECT a.user, COUNT(*) as num
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) a
GROUP BY a.user
ORDER BY num DESC
LIMIT 10;
```

```
Teddy73|697934
jotpe|323776
catweazle67|95236
cgn1234|86072
okilimu|84968
Grauer|56736
berndw|48777
Raymond|43883
phanaton|35897
WoGraSo|34371
```

### Most popular drug store

The two most popular drug stores in Germany are dm-drogerie Markt and Rossmann. To confirm this, it was first necessary to make all the dm-drogerie Markt entries have consistent value for the 'name' key. Since this was not done programatically in the cleaning stage, it was done via an update to the nodes_tags table, after having collected the ids of the nodes that needed the update.
```SQL
sqlite> UPDATE nodes_tags SET value = 'dm' WHERE key = 'name' AND id IN (382026590, 498764996, 518836335, 571870877, 603711673, 659594930, 678884929, 690255494, 693300582, 698868701, 2141236631, 2439165241, 2604449704, 3220016064, 3376705901);```

```SQL
sqlite> SELECT a.value, COUNT(a.id) as count
FROM nodes_tags a
    JOIN (SELECT id FROM nodes_tags WHERE value = 'chemist') b
    ON a.id = b.id
WHERE a.key = 'name'
GROUP BY a.value
ORDER BY count DESC;```

```
dm|27
Rossmann|22
Augenärztliche Gemeinschaftspraxis|1
EuroShop Filiale Köln|1
Foto Team & Reformhaus Gymnich|1
German Style Healthcare|1
KODI|1
Kiehl's|1
Küpper Höpfner|1
PM|1
The Body Shop|1```
### Religion
![alt text](https://raw.githubusercontent.com/DiogoAdaoeSilva/DAND/master/p3/cologne_cathedral.jpg "Von © Raimond Spekking / CC BY-SA 4.0 (via Wikimedia Commons), CC BY-SA 4.0")
<p>Cologne is know for its beautiful [gothic cathedral](https://en.wikipedia.org/wiki/Cologne_Cathedral) and catholic tradition. How is the distribution of places of worship in the city like?
```SQL
sqlite> SELECT ways_tags.value, COUNT(*) as num
FROM ways_tags
    JOIN (SELECT DISTINCT(id) FROM ways_tags WHERE value='place_of_worship') a
    ON ways_tags.id=a.id
WHERE ways_tags.key='religion'
GROUP BY ways_tags.value
ORDER BY num DESC;```
```
christian|286
muslim|25
jewish|3
sikh|3
hindu|2```
The result matches my expectations based on what I observed while living in Cologne.
### Christian denominations

```SQL
sqlite> SELECT b.value, COUNT(*) as num
FROM ways_tags
    JOIN (SELECT DISTINCT(id) FROM ways_tags WHERE value='place_of_worship') a
    ON ways_tags.id=a.id
    JOIN (SELECT DISTINCT(id), value FROM ways_tags WHERE key = 'denomination') b
    ON a.id = b.id 
WHERE ways_tags.key='religion' AND ways_tags.value = 'christian'
GROUP BY b.value
ORDER BY num DESC
LIMIT 10;```


```
catholic|116
protestant|69
roman_catholic|31
new_apostolic|6
evangelical|4
jehovahs_witness|4
lutheran|2
pentecostal|2
russian_orthodox|2
anglican|1
```

## Additional ideas
-----
Ensuring consistency in user generated data is a complex challenge. My personal experince with editing OSM, showed me how easy it is to enter data in the wrong format. I entered a local business in Cologne, including its phone number. Only later when I reached the auditing stage of this project did I realize I had entered the wrong phone number format.

One possible way to address this would be for first-time users of OSM to see tool tips showing up when editing fields. This way a new user could see the suggested formats and enter new data correctly. This could be applied to many fields such as cuisine, city and religious denominations.

## Conclusion
-----
The Open Street Map of the city region of Cologne, Germany, was audited, in particular the fields street, postcode and phone. For the latter two, problems were encountered and cleaned programatically. Later, the data was converted from XML to tabular format and imported to a SQL database. Querying the database allowed to have a statistical overview of the dataset and also, to find new problems with the dataset and identify an error made earlier in the cleaning stage. Although the dataset is in general complete, valid and accurate, its quality regarding consistency can be improved.